In [5]:
import matplotlib.pyplot as plt

import numpy as np

 

Inf = None

 

def modinv(a, m):

    m0, x0, x1 = m, 0, 1

    while a > 1:

        q = a // m

        m, a = a % m, m

        x0, x1 = x1 - q * x0, x0

    return x1 + m0 if x1 < 0 else x1

 

def ECC_valid(a, b, p):

    """

    Check if the given elliptic curve is singular.

 

    Parameters:

    - a (int): Coefficient 'a' in the elliptic curve equation

    - b (int): Coefficient 'b' in the elliptic curve equation

    - p (int): Prime number

 

    Returns:

    bool: True if the elliptic curve is non-singular, False otherwise

    """

    return (4 * (a**3) + 27 * (b**2)) % p != 0

 

def ECC_genpts(a, b, p):

    """

    Generate and return a list of all points in the modular elliptic curve.

 

    Parameters:

    - a (int): Coefficient 'a' in the elliptic curve equation

    - b (int): Coefficient 'b' in the elliptic curve equation

    - p (int): Prime number

 

    Returns:

    list: List of points in the elliptic curve in the form (x, y)

    """

    points = []

 

    for x in range(p):

        y_squared = (x**3 + a * x + b) % p

        y = None

 

        # Check if y^2 is a quadratic residue modulo p

        if pow(y_squared, (p-1)//2, p) == 1:

            y = pow(y_squared, (p+1)//4, p)

            points.append((x, y))

            points.append((x, p - y))  # Add the other solution

 

    points.append(Inf)  # Include the point at infinity

    return points

 

def ECC_plotPxy(a, b, p):

    """

    Plot the points in the modular elliptic curve.

 

    Parameters:

    - a (int): Coefficient 'a' in the elliptic curve equation

    - b (int): Coefficient 'b' in the elliptic curve equation

    - p (int): Prime number

    """

    points = ECC_genpts(a, b, p)

 

    # Filter out the point at infinity for plotting

    valid_points = [point for point in points if point != Inf]

 

    x_vals = [point[0] for point in valid_points]

    y_vals = [point[1] for point in valid_points]

 

    plt.scatter(x_vals, y_vals, label='Points in E')

    plt.title(f'Modular Elliptic Curve: E: y^2 = x^3 + {a}x + {b} (mod {p})')

    plt.xlabel('x')

    plt.ylabel('y')

    plt.legend()

    plt.grid(True)

    plt.show()

 

# # Example usage:

# a = 2

# b = 2

# p = 17

# ECC_plotPxy(a, b, p)

 

def ECC_pltline(P,a,b,p):

    x1=P[0]

    y1=P[1]

    P1=P

    ECC_plotPxy(a,b,p)

    while(P1!='Inf'):# and ECC_isPinE(P1, a, b, p)):

        P1=ECC_addPQ(P,P1,a,b,p)

        if(P1=='Inf'):

            break

        if(not ECC_isPinE(P1,a,b,p)):

            break

        if(ECC_isPinE(P1, a, b, p)):

            print(P1, "is in E")

            plt.plot([x1,P1[0]],[y1,P1[1]],linestyle='solid',marker='o',color='green')

        x1=P1[0]

        y1=P1[1]



 

def ECC_orderP(P, a, b, p):

    """

    Given P(x, y) in E, find the order of P.

 

    Parameters:

    - P (tuple): Point P in the elliptic curve in the form (x, y)

    - a (int): Coefficient 'a' in the elliptic curve equation

    - b (int): Coefficient 'b' in the elliptic curve equation

    - p (int): Prime number

 

    Returns:

    int: Order of P

    """

    # This can be completed only after

    # modifying the fn: for adding P & Q

    # to handle the add: identity properly

    ECC_pltline(P, a, b, p)

    i = 1

    if ECC_isPinE(P, a, b, p):

        P1 = P

        while P1 != Inf:

            print(i, P1)

            P1 = ECC_addPQ(P, P1, a, b, p)

            if P1 is None:

                break

            if not ECC_isPinE(P1, a, b, p):

                break

            i += 1

        return i

 

def ECC_isPinE(P, a, b, p):

    """

    Return True if P belongs to E (elliptic curve), or False.

 

    Parameters:

    - P (tuple): Point P in the elliptic curve in the form (x, y)

    - a (int): Coefficient 'a' in the elliptic curve equation

    - b (int): Coefficient 'b' in the elliptic curve equation

    - p (int): Prime number

 

    Returns:

    bool: True if P belongs to E, False otherwise

    """

    # y2 = x3 + ax + b ( mod p )

    x = P[0]

    y = P[1]

    lhs = (y*y)%p

    rhs = (x*x*x + a*x + b)%p

    if(lhs==rhs):

        return True

    else:

        return False

 

def ECC_addPQ(P, Q, a, b, p):

    """

    Return the coordinates of P(x1, y1) + Q(x2, y2).

 

    Parameters:

    - P (tuple): Point P in the elliptic curve in the form (x1, y1)

    - Q (tuple): Point Q in the elliptic curve in the form (x2, y2)

    - a (int): Coefficient 'a' in the elliptic curve equation

    - b (int): Coefficient 'b' in the elliptic curve equation

    - p (int): Prime number

 

    Returns:

    tuple: Coordinates of P + Q in the form (x3, y3)

    """

    if P == Inf:

        return Q

    if Q == Inf:

        return P

 

    x1, y1 = P

    x2, y2 = Q

 

    if P == Q:

        # Point doubling

        m = (3 * x1**2 + a) * modinv(2 * y1, p) % p

    else:

        # Point addition

        m = (y2 - y1) * modinv(x2 - x1, p) % p

 

    x3 = (m**2 - x1 - x2) % p

    y3 = (m * (x1 - x3) - y1) % p

 

    return (x3, y3)

 

def ECC_nP(n, P, a, b, p):

    """

    Return the coordinates of P + ... + P, i.e., P added n times to itself.

 

    Parameters:

    - n (int): Number of times to add P to itself

    - P (tuple): Point P in the elliptic curve in the form (x, y)

    - a (int): Coefficient 'a' in the elliptic curve equation

    - b (int): Coefficient 'b' in the elliptic curve equation

    - p (int): Prime number

 

    Returns:

    tuple: Coordinates of P + ... + P in the form (x, y)

    """

    result = Inf

    for _ in range(n.bit_length()):

        if n & 1:

            result = ECC_addPQ(result, P, a, b, p)

        P = ECC_addPQ(P, P, a, b, p)

        n >>= 1

    return result

 

def ECC_2nP(n, P, a, b, p):

    """

    Return the coordinates of P + ... + P, i.e., P added n times to itself where n is a power of 2.

 

    Parameters:

    - n (int): Power of 2, indicating the number of times to add P to itself

    - P (tuple): Point P in the elliptic curve in the form (x, y)

    - a (int): Coefficient 'a' in the elliptic curve equation

    - b (int): Coefficient 'b' in the elliptic curve equation

    - p (int): Prime number

 

    Returns:

    tuple: Coordinates of P + ... + P in the form (x, y)

    """

    result = P

    for _ in range(n.bit_length() - 1):

        result = ECC_addPQ(result, result, a, b, p)

    return result

 

# Example usage:

a = 2

b = 2

p = 17

ECC_plotPxy(a, b, p)

P = (5, 1)

Q = (5, 16)

R = ECC_addPQ(P, Q, a, b, p)

print(R)

print('\n')

order = ECC_orderP(P, a, b, p)

print(order)

print('\n')

is_in_curve = ECC_isPinE(P, a, b, p)

print(is_in_curve)

print('\n')

nP = ECC_nP(3, P, a, b, p)

print(nP)

print('\n')

two_nP = ECC_2nP(4, P, a, b, p)

print(two_nP)

ZeroDivisionError: integer division or modulo by zero